In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms,models
from torchvision.transforms import Resize, ToTensor, Normalize
from PIL import Image
from torch.utils.data.dataset import random_split

In [2]:
train_dir = '/opt/ml/input/data/train/images'
transform = transforms.Compose([
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
]) 


In [3]:
class maskImageDataset(Dataset):
    def __init__(self,path,transform=None):
        self.image,self.label = self.labeling(path)
        self.transform = transform
        
    def __getitem__(self,idx):
        image,label = Image.open(self.image[idx]),self.label[idx]
        if self.transform:
            image = self.transform(image)
        return image,label

    def __len__(self):
            return len(self.label)

    def labeling(self,paths):
        x = []
        y = []
        for dic in os.listdir(paths):
            if '._' in dic or 'ipynb_checkpoints' in dic:
                continue
            dir_path = paths + '/'+ dic
            code = 0
            if dic[7] == 'f':
                code = 3
            age = int(dic[-2:])
            if age >= 60:
                code += 2
            elif age >=30:
                code += 1
            for image in os.listdir(dir_path):
                if '._' in image or 'ipynb_checkpoints' in image:
                    continue
                image_path = dir_path + '/' + image 
                x.append(image_path)
                label = [0 for _ in range(18)]
                y.append(self.age_labeling(image,code))
        return x,y
                
    def age_labeling(self,path,inputs):
        if 'incorrect_mask' in path:
            return inputs + 6
        elif 'normal' in path:
            return inputs + 12
        else:
            return inputs

In [16]:
DATA = maskImageDataset(train_dir,transform)

train_length=int(0.8* len(DATA))
test_length=len(DATA)-train_length



train_DATA,val_DATA=random_split(DATA,(train_length,test_length))
train_loader = DataLoader(dataset=train_DATA, batch_size=16,shuffle=True)
val_loader   = DataLoader(dataset=val_DATA, batch_size=20)


In [17]:
resnet50=models.resnet50(pretrained=True)


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(f"{device} is using!")
resnet50.to(device)
LEARNING_RATE = 0.0001
NUM_EPOCH = 5 
loss_fn = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(resnet50.parameters(), lr=LEARNING_RATE)


cuda:0 is using!


In [11]:
from tqdm.notebook import tqdm


In [19]:
resnet50.fc=torch.nn.Linear(in_features=2048, out_features=18, bias=True)
torch.nn.init.xavier_uniform_(resnet50.fc.weight)
stdv = 1/len(DATA) 
resnet50.fc.bias.data.uniform_(-stdv, stdv)
if torch.cuda.is_available():
    resnet50.cuda()

In [20]:
dataloaders = {
    "train" : train_loader,
    "val":val_loader
}


In [21]:
best_test_accuracy=0
best_test_loss=9999.
for epoch in range(5):
    for phase in ["train","val"]:
        running_loss=0.
        running_acc=0.
        if phase == "train":
            resnet50.train()
        if phase == "val":
            resnet50.eval()
        
        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
            images=images.to(device)
            labels=labels.to(device)

            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=="train"):
                logits=resnet50(images)
                _,preds=torch.max(logits,1)
                loss=loss_fn(logits,labels)
            if phase =="train":
                loss.backward()
                optimizer.step()
        
            running_loss+=loss.item()*images.size(0)
            running_acc+=torch.sum(preds==labels.data)
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: 
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss:
      best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")




현재 epoch-0의 val-데이터 셋에서 평균 Loss : 0.296, 평균 Accuracy : 0.903



현재 epoch-1의 val-데이터 셋에서 평균 Loss : 0.182, 평균 Accuracy : 0.939



현재 epoch-2의 val-데이터 셋에서 평균 Loss : 0.095, 평균 Accuracy : 0.968



현재 epoch-3의 val-데이터 셋에서 평균 Loss : 0.067, 평균 Accuracy : 0.980



현재 epoch-4의 val-데이터 셋에서 평균 Loss : 0.070, 평균 Accuracy : 0.980
학습 종료!
최고 accuracy : 0, 최고 낮은 loss : 9999.0


TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not list

In [27]:
PATH = './'
torch.save(resnet50,PATH+'model.pt')